In [1]:
import pandas as pd
from ast import literal_eval
import re

In [ ]:
def remove_parentheses(text):
    cleaned_text = re.sub(r'\([^)]*\)', '', text).strip()
    return cleaned_text.split(' ; ')
def split_law_and_article(text):
    # '제숫자조' 뒤에 의2와 같은 추가 내용을 포함하도록 정규식 수정
    match = re.match(r'(.*?)(제\d+조[\w]*)', text)
    if match:
        law_name = match.group(1).replace(' ', '').strip()  # 법 이름 부분
        article = match.group(2).strip()  # 조항 부분
        return [law_name, article]
    else:
        return [text.strip(), '']
# Shuffle and split the dataframe 6 times
test_df = pd.read_csv('/home/kyeonghyun/test_retrieval/materials/민원22~24년도통합_partial_drop_yoji.csv', encoding='utf-8-sig')
test = test_df
test_df['추출된 법(re)'] = test_df['추출된 법(re)'].apply(literal_eval)
test_df['추출된 법(split)'] = test_df['추출된 법(re)'].apply(lambda x: [split_law_and_article(law) for law in x])
test_df['추출된법_law_clause'] = test_df['추출된 법(split)'].apply(lambda x: [law[1] for law in x])


In [ ]:
test_df['추출된 법(re)'][1]

['건축법 시행령 제3조의5', '건축법 제2조', '건축법 시행령 제2조']

In [21]:
test_df['추출된 법(split)'][1]

[['건축법시행령', '제3조의5'], ['건축법', '제2조'], ['건축법시행령', '제2조']]

In [17]:
test_df['추출된법_law_clause'][1]

['제3조의5', '제2조', '제2조']

In [3]:
test_unique_minwon_list = []
for k in range(len(test_df)):
    index_list = []
    for law in range(len(test_df['추출된법_law_clause'][k])):
        if '제1조' in test_df['추출된법_law_clause'][k][law] :
            index_list.append(law)
        elif '제2조' in test_df['추출된법_law_clause'][k][law] and '제2조의' not in test_df['추출된법_law_clause'][k][law]:
            index_list.append(law)
    test_unique_minwon_list.append(index_list)

        


In [16]:
test_unique_minwon_list[1]

[1, 2]

In [22]:
for idx in range(len(test_unique_minwon_list)):
    if test_unique_minwon_list[idx] == 0:
        pass
    else:
        list_ = sorted(test_unique_minwon_list[idx], reverse=True)
        law_list = test_df['추출된 법(re)'][idx]
        for k in list_:
            del law_list[k]
        test_df.at[idx, '추출된 법(re)'] = law_list   
test_df = test_df[test_df['추출된 법(re)'].apply(len) != 0]
test_df = test_df.reset_index(drop=True)  
test_df = test_df.drop(['추출된 법(split)', '추출된법_law_clause'], axis=1)

In [23]:
test_df['추출된 법(re)'][1]

['건축법 시행령 제3조의5']

In [ ]:


if test['민원내용'][0] == test_df['민원내용'][0]:
    pass
else:
    print('error')

test_df.to_csv(f'/home/kyeonghyun/test_retrieval/data1/민원22~24년도통합_test_partial_drop_no12.csv', index=False, encoding='utf-8-sig')
